<a href="https://colab.research.google.com/github/Ivanbh214/Machine_learning_Course_CIFO/blob/main/D%C3%ADa_14(15-11-21)/XarxaNeuronalSimple1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from math import exp
from copy import copy, deepcopy
from random import random

def f_sigmoidal(x, a = 1.0, x0 = 0.0): # Funcio logistica
    return 1. / (1. + exp(- a * (x - x0)))

class Neurona:
    def __init__(self, Id, IdCapa, pesos, w0, f_activacio, receptores):
        self.Id = Id
        self.IdCapa = IdCapa  # Capa d'entrada = Capa 0
        self.pesos = pesos
        self.w0 = w0
        self.f_activacio = f_activacio
        self.receptores = receptores
        self.sortidaCorrecta = None # Nomes per neurones de la capa de sortida
    def senyalsEntrada(self, xarxa): # Senyals que rep aquesta neurona
        assert self.IdCapa > 0 # Per assegurar que no es la capa d'entrada
        entrades = {}
        capa_anterior = xarxa.capes[self.IdCapa - 1]
        for neurona in capa_anterior.neurones:
            if self.Id in neurona.receptores:
                entrades[neurona.Id] = neurona.senyalSortida(xarxa)
        return entrades  # Diccionari: Id de neurona -> Senyal d'aquella neurona
    def senyalSortida(self, xarxa): # Senyal que emet aquesta neurona
        suma_ponderada = self.w0
        if self.IdCapa == 0: return suma_ponderada # Capa d'entrada: w0 = senyal
        else:
            entrades = self.senyalsEntrada(xarxa)
            for emissora in entrades:
                suma_ponderada += entrades[emissora] * self.pesos[emissora]
            return self.f_activacio(suma_ponderada)
    def senyalSortidaCorrecte(self, valor):
        self.sortidaCorrecta = valor # Nomes per neurones de la capa de sortida

    # Es pot fer mes eficient si delta es un atribut actualizable de la neurona
    def delta(self, xarxa): # f_activacio = f_sigmoidal, f_error = f_quadratica
        o = self.senyalSortida(xarxa)
        if self.esNeuronaDeSortida(xarxa):
            t = self.sortidaCorrecta
            return (o - t) * o * (1 - o)
        else:
            suma_ponderada_deltes = 0.0
            neurones_capa_posterior = xarxa.capes[self.IdCapa + 1].neurones
            for receptora in neurones_capa_posterior:
                if receptora.Id in self.receptores:
                    suma_ponderada_deltes += receptora.delta(xarxa) * receptora.pesos[self.Id]
            return suma_ponderada_deltes * o * (1 - o)
    def variacioPesos(self, xarxa):
        incrementPesos = {}
        entrades = self.senyalsEntrada(xarxa)
        for Id in entrades:
            incrementPesos[Id] = -xarxa.ritme * entrades[Id] * self.delta(xarxa)
        return incrementPesos
    def aplicaRetropropagacio(self, xarxa):
        incrementPesos = self.variacioPesos(xarxa)
        for Id in incrementPesos:
            self.pesos[Id] += incrementPesos[Id]
    def iniciAleatori(self, pes_min, pes_max, w0_min, w0_max):
        self.w0 = w0_min + (w0_max - w0_min) * random()
        for Id in self.pesos:
            self.pesos[Id] = pes_min + (pes_max - pes_min) * random()
    def esNeuronaDeSortida(self, xarxa):
        return self.IdCapa == max(xarxa.capes)
    def actualitza_w0(self, valor): # Nomes per la capa d'entrada
        assert self.IdCapa == 0     # Comprova que es la capa d'entrada
        self.w0 = valor
    def mostra(self, mostraCapa = False):
        print("Neurona:", self.Id)
        if mostraCapa: print("Capa:", self.IdCapa)
        print("Pesos:", self.pesos)
        print("w0:", self.w0)
        print("Funcio d'activacio:", self.f_activacio)
        print("Neurones receptores:", self.receptores)

class Capa:
    def __init__(self, IdCapa, neurones):
        self.IdCapa = IdCapa
        self.neurones = neurones
        for neurona in self.neurones:
            neurona.IdCapa = IdCapa
    def clonada(self, novaId):
        return Capa(novaId, [deepcopy(neurona) for neurona in self.neurones])
    def mostra(self):
        if self.IdCapa == 0: print("Capa:", self.IdCapa, "(capa d'entrada)")
        else: print("Capa:", self.IdCapa)
        for neurona in self.neurones:
            neurona.mostra()
    def senyalSortida(self, xarxa):
        return [neurona.senyalSortida(xarxa) for neurona in self.neurones]
    def senyalSortidaCorrecte(self, valors): # Nomes per la capa de sortida
        for neurona, valor in zip(self.neurones, valors):
            neurona.senyalSortidaCorrecte(valor)
    def senyalEntrada(self, valors): # Nomes per la capa d'entrada
        assert self.IdCapa == 0 # Comprova que es la capa d'entrada
        for neurona, valor in zip(self.neurones, valors):
            neurona.w0 = valor
    def aplicaRetropropagacio(self, xarxa):
        for neurona in self.neurones:
            neurona.aplicaRetropropagacio(xarxa)
    def iniciAleatori(self, pes_min, pes_max, t_i_min, t_i_max):
        for neurona in self.neurones:
            neurona.iniciAleatori(pes_min, pes_max, t_i_min, t_i_max)

class Xarxa:
    def __init__(self, capa_entrada, llista_capes, ritme):
        self.capes = {}
        self.capes[0] = capa_entrada
        for capa in llista_capes:
            assert capa.IdCapa > 0
            self.capes[capa.IdCapa] = capa
        self.ritme = ritme
    def mostra(self):
        for IdCapa in self.capes:
            self.capes[IdCapa].mostra()
            print("")
    def capaDeSortida(self):
        maxId = max(self.capes)
        return self.capes[maxId]
    def senyalSortida(self, valorsEntrada):
        self.capes[0].senyalEntrada(valorsEntrada)
        return self.capaDeSortida().senyalSortida(self)
    def senyalSortidaCorrecte(self, valorsSortida):
        self.capaDeSortida().senyalSortidaCorrecte(valorsSortida)
    def aplicaRetropropagacio(self, valorsSortida, valorsEntrada):
        self.senyalSortidaCorrecte(valorsSortida)
        self.capes[0].senyalEntrada(valorsEntrada)
        #llistaDescendentIdentificadors = sorted(self.capes.keys(), reverse = 1)
        llistaDescendentIdentificadors = sorted(self.capes.keys())[::-1]
        for IdCapa in llistaDescendentIdentificadors[:-1]:
            self.capes[IdCapa].aplicaRetropropagacio(self)
    def iniciAleatori(self, pes_min, pes_max, t_i_min, t_i_max):
        for capa in self.capes.values():
            if capa.IdCapa > 0:
                capa.iniciAleatori(pes_min, pes_max, t_i_min, t_i_max)


n1 = Neurona(1, None, {1: 1.0, 2: 2.0, 3: -0.5}, 0.0, f_sigmoidal, [1, 2, 3])
n2 = Neurona(2, None, {1: 1.0, 2: 1.0, 3: -1.5}, 0.0, f_sigmoidal, [1, 2, 3])
n3 = Neurona(3, None, {1: 1.0, 2: 0.0, 3: +0.5}, 0.0, f_sigmoidal, [1, 2, 3])

c0 = Capa(0, [n1, n2, n3])
c1 = c0.clonada(1)
c2 = c0.clonada(2)
c3 = c0.clonada(3)


X = Xarxa(c0, [c1, c2, c3], 1.0) #1.0 ritmo de aprendizaje
#Dependiendo de la función que se utilice si son valores bastantes grandes los datos se deben normalizar

#X.mostra()
X.iniciAleatori(-1.0, 1.0, -1.0, 1.0)#Estos valores son aleatorios
#X.mostra()

print(X.senyalSortida([0.0, 0.5, -0.5]))

print("***")

for i in range(1000):
    X.aplicaRetropropagacio([0.2, 0.3, 0.4], [0.5, -0.5, 0.5])
    if i % 10 == 0: print(X.senyalSortida([0.5, -0.5, 0.5]))

